# Batch EQL Queries

In [1]:
import os
import inspect
import time
import math
import json
import logging
import pickle

from energyworx_client.client import EWX
from energyworx_client import storage

import pandas as pd
import numpy as np
from scipy import stats
import pprint
import datetime as dt

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

from IPython.display import display


base = os.getcwd()
namespace = 'na.engie.com'
api = EWX(namespace)
pp = pprint.PrettyPrinter(indent = 1)

# Function to Batch Large Queries

In [2]:
def single_query(query, lim, job, pg, result_data):
    
    if job and pg:
        meta_data  = api.execute_query(query, limit = lim, raw_result = True, job_id = job, page_token = pg) 
    
    else:
        meta_data  = api.execute_query(query, limit = lim, raw_result = True)
    
    try:
        pt = str(meta_data['pageToken']) #get page token
        tot = int(meta_data['metadata']['totalRows']) #get total rows
        job = str(meta_data['reference']['jobId']) #get job id
        
    except:
        print 'no page token needed.'

    var_fields = [] #get var names

    for f in meta_data['metadata']['fields']:
        var_fields.append(str(f['field']))
        
    p = len(var_fields)
        
    for row in meta_data['rows']:
        try:
            final_row = []
            for r in row['f']:
                final_row.append(str(r['v']))
                
            result_data.append(final_row)
            
        except:
            print 'row error'
        
        
    return(result_data, tot, pt, job, var_fields)

def large_query(query, num):
    
    tot_start = time.time()
    result = []
    print 'starting query...'
    start = time.time()
    result, n, pt, job, var = single_query(query, num, None, None, result)
    end = time.time()
    m = len(result)
    i = 0
    elapse = round(end - start, 2)
    print 'done with {} out of {} rows in {} seconds.'.format(m, n, elapse)
    
    batch = int(math.ceil(n/num))
    print 'running {} more batches:'.format(batch)
    
    while i < batch and batch > 0:
        try:
            start = time.time()
            print 'running batch {}...'.format(i+1)
            
            result, n, pt, job, var = single_query(query, num, job, pt, result)
            batch = int(math.ceil(n/num))
            
            end = time.time()
            elapse = round(end - start, 2)
            m = len(result)
            
            print 'done with {} out of {} rows in {} seconds,'.format(m, n, elapse)
            i += 1
            print 'ran batch {} of {}.'.format(i, batch)
            
            if (n - m) < num:
                num = n - m
        
        except:
            print 'error (hopefully out of rows to query).'
            i += 1
    
    tot_end = time.time()
    print 'query finished in {} total seconds.'.format(round(tot_end-tot_start, 2))
    return(result, var)
    

## Metadata Query

In [7]:
meta_query = "WITH PRlist AS (SELECT DISTINCT(datasource_id) FROM tags, UNNEST (properties) props WHERE props.key = 'prnumber' AND props.value = '1-IFF3XL'),\
stats AS (select datasource_id,flow_timestamp,flow_type,channel_classifier_id,flow_id, p.key, p.value from flow_metadata, UNNEST(PROPERTIES) AS p),\
MaxTimeStamp as (select datasource_id, MAX(flow_timestamp) m_flow_timestamp from flow_metadata GROUP BY datasource_id)\
Select s.* from stats AS s,PRlist AS prl,MaxTimeStamp as mtsp WHERE \
(s.key ='datasetMetadata.RAW.startDate' OR s.key = 'datasetMetadata.RAW.endDate' or s.key = 'datasetMetadata.RAW.usage' or s.key = 'datasetMetadata.MERGED.usage' or s.key ='datasetMetadata.MERGED.startDate' OR s.key = 'datasetMetadata.MERGED.endDate') \
and s.datasource_id IN (prl.datasource_id) and s.flow_timestamp IN (mtsp.m_flow_timestamp)"

output, variables = large_query(meta_query, 500)

starting query...
done with 500 out of 3090 rows in 55.65 seconds.
running 6 more batches:
running batch 1...
done with 1000 out of 3090 rows in 47.9 seconds,
ran batch 1 of 6.
running batch 2...
done with 1500 out of 3090 rows in 21.54 seconds,
ran batch 2 of 6.
running batch 3...
done with 2000 out of 3090 rows in 39.71 seconds,
ran batch 3 of 6.
running batch 4...
done with 2500 out of 3090 rows in 20.41 seconds,
ran batch 4 of 6.
running batch 5...
done with 3000 out of 3090 rows in 20.26 seconds,
ran batch 5 of 6.
running batch 6...
no page token needed.
error (hopefully out of rows to query).
query finished in 210.03 total seconds.


## Test smaller batch size

In [8]:
output, variables = large_query(meta_query, 50)

starting query...
done with 50 out of 3090 rows in 18.47 seconds.
running 61 more batches:
running batch 1...
done with 100 out of 3090 rows in 5.56 seconds,
ran batch 1 of 61.
running batch 2...
done with 150 out of 3090 rows in 2.73 seconds,
ran batch 2 of 61.
running batch 3...
done with 200 out of 3090 rows in 4.33 seconds,
ran batch 3 of 61.
running batch 4...
done with 250 out of 3090 rows in 2.93 seconds,
ran batch 4 of 61.
running batch 5...
done with 300 out of 3090 rows in 4.29 seconds,
ran batch 5 of 61.
running batch 6...
done with 350 out of 3090 rows in 2.81 seconds,
ran batch 6 of 61.
running batch 7...
done with 400 out of 3090 rows in 2.64 seconds,
ran batch 7 of 61.
running batch 8...
done with 450 out of 3090 rows in 2.95 seconds,
ran batch 8 of 61.
running batch 9...
done with 500 out of 3090 rows in 7.48 seconds,
ran batch 9 of 61.
running batch 10...
done with 550 out of 3090 rows in 3.52 seconds,
ran batch 10 of 61.
running batch 11...
done with 600 out of 3090 r

## Convert output to dataframe

In [6]:
output_df = pd.DataFrame(output)
output_df.columns = variables
output_df['key'] = ['_'.join(k.split('.')[1:]) for k in output_df.key]

                      datasource_id       flow_timestamp flow_type  \
0    ERCOT_AEPCPL_10032789489124369  1.550676853554602E9  scenario   
1     ERCOT_ONCOR_10443720000474400  1.550677353939528E9  scenario   
2  ERCOT_CPT_1008901016900570530112  1.550681244868334E9  scenario   
3  ERCOT_CPT_1008901007114120287100  1.550681951545643E9  scenario   
4  ERCOT_CPT_1008901023804689270100  1.550682340670881E9  scenario   

  channel_classifier_id                           flow_id        key     value  
0                MERGED  c2af7f651e4844a79d6d764697bca80e  RAW_usage  188889.0  
1                MERGED  9ceb655319674d2f87f0cd1beffab46a  RAW_usage  116880.0  
2                MERGED  bd1f2e9211db4c70aeee6f596d64e55a  RAW_usage  210560.0  
3                MERGED  ea033866ca04437799be8092b194c2ad  RAW_usage  409480.0  
4                MERGED  e7cd071817284cd98be4a35b43c9e12b  RAW_usage  592480.0  
...
                         datasource_id       flow_timestamp flow_type  \
3085  ERCOT_CPT_

## Pivot dataframe, type conversion

In [8]:
final_output = output_df.pivot(index = 'datasource_id', columns = 'key', values = 'value')

final_output['MERGED_endDate'] = pd.to_datetime(final_output.MERGED_endDate)
final_output['MERGED_startDate'] = pd.to_datetime(final_output.MERGED_startDate)
final_output['RAW_endDate'] = pd.to_datetime(final_output.RAW_endDate)
final_output['RAW_startDate'] = pd.to_datetime(final_output.RAW_startDate)
final_output['MERGED_usage'] = pd.to_numeric(final_output.MERGED_usage)
final_output['RAW_usage'] = pd.to_numeric(final_output.RAW_usage)
## time period units = days
final_output['MERGED_timePd'] = [round((pd.Timedelta(e - s).total_seconds() / 86400), 8) for e, s in zip(final_output.MERGED_endDate, final_output.MERGED_startDate)]
final_output['RAW_timePd'] = [round((pd.Timedelta(e - s).total_seconds() / 86400), 8) for e, s in zip(final_output.RAW_endDate, final_output.RAW_startDate)]
final_output['MERGED_meanUse'] = [u / t for u, t in zip(final_output.MERGED_usage, final_output.MERGED_timePd)]
final_output['RAW_meanUse'] = [u / t for u, t in zip(final_output.RAW_usage, final_output.RAW_timePd)]
final_output.head()

key,MERGED_endDate,MERGED_startDate,MERGED_usage,RAW_endDate,RAW_startDate,RAW_usage,MERGED_timePd,RAW_timePd,MERGED_meanUse,RAW_meanUse
datasource_id,,,,,,,,,,
ERCOT_AEPCPL_10032789400513438,2029-11-01 05:00:00,2016-05-23 05:00:00,1.681069e+06,2019-01-23 06:00:00,2016-05-23 05:00:00,245073.0,4910.0,975.041667,342.376628,251.346182
ERCOT_AEPCPL_10032789400766221,2029-11-01 05:00:00,2016-05-09 05:00:00,3.666146e+06,2019-02-07 06:00:00,2016-05-09 05:00:00,572320.0,4924.0,1004.041667,744.546334,570.016185
ERCOT_AEPCPL_10032789400766222,2029-11-01 05:00:00,2016-05-09 05:00:00,2.016774e+05,2019-02-07 06:00:00,2016-05-09 05:00:00,32240.0,4924.0,1004.041667,40.958039,32.110221
ERCOT_AEPCPL_10032789401211235,2029-11-01 05:00:00,2016-04-29 05:00:00,1.955844e+06,2019-01-30 06:00:00,2016-04-29 05:00:00,292753.0,4934.0,1006.041667,396.401346,290.994906
ERCOT_AEPCPL_10032789401957901,2029-11-01 05:00:00,2016-05-05 05:00:00,2.557878e+04,2019-02-05 06:00:00,2016-05-05 05:00:00,4030.0,4928.0,1006.041667,5.190500,4.005798


In [9]:
final_output.shape

(515, 10)

In [10]:
storage.to_csv(final_output, 'metadata_query_output.csv')

## Geomapping

In [4]:
geomap = "WITH dvm AS (SELECT distinct datasource_id AS dvm_datasource_id, MAX(version) dvm_version FROM tags, UNNEST(properties) AS p WHERE tag = 'geopoint' GROUP BY datasource_id),\
dvm1 AS (SELECT distinct datasource_id AS dvm_datasource_id, MAX(version) dvm_version FROM tags, UNNEST(properties) AS p WHERE tag = 'premise' GROUP BY datasource_id),\
latitudes AS (SELECT distinct datasource_id, version AS lat_version, p.value as latitude FROM tags, UNNEST(properties) AS p WHERE tag = 'geopoint' AND p.key = 'latitude'),\
longitudes AS (SELECT distinct datasource_id, version AS long_version, p.value as longitude FROM tags, UNNEST(properties) AS p WHERE tag = 'geopoint' AND p.key = 'longitude'),\
zips AS (SELECT distinct datasource_id, version AS zip_version, p.value as zip FROM tags, UNNEST(properties) AS p WHERE tag = 'premise' AND p.key = 'zip'),\
fl AS (SELECT distinct datasource_id, MAX(flow_timestamp) AS tsmp from flow_metadata GROUP BY datasource_id ORDER by tsmp) \
SELECT distinct fl.datasource_id, DATETIME(fl.tsmp, 'America/Chicago') AS flow_tmsp, zips.zip, DATETIME(zips.zip_version, 'America/Chicago') AS premise_version, \
latitudes.latitude, longitudes.longitude, DATETIME(latitudes.lat_version, 'America/Chicago') AS geopoint_version \
FROM fl \
INNER JOIN zips on zips.datasource_id = fl.datasource_id \
INNER JOIN latitudes on latitudes.datasource_id = fl.datasource_id \
INNER JOIN longitudes on longitudes.datasource_id = fl.datasource_id \
INNER JOIN dvm on dvm.dvm_datasource_id = fl.datasource_id \
INNER JOIN dvm1 on dvm1.dvm_datasource_id = fl.datasource_id \
WHERE fl.tsmp > '2019-07-19T00:00:00' and fl.tsmp <= '2019-07-19T23:00:00' \
AND dvm.dvm_version = latitudes.lat_version \
AND dvm.dvm_version = longitudes.long_version \
AND dvm1.dvm_version = zips.zip_version"

In [5]:
geo, geo_var = large_query(geomap, 50)

starting query...
done with 50 out of 434 rows in 19.41 seconds.
running 8 more batches:
running batch 1...
done with 100 out of 434 rows in 4.27 seconds,
ran batch 1 of 8.
running batch 2...
done with 150 out of 434 rows in 4.51 seconds,
ran batch 2 of 8.
running batch 3...
done with 200 out of 434 rows in 4.15 seconds,
ran batch 3 of 8.
running batch 4...
done with 250 out of 434 rows in 6.52 seconds,
ran batch 4 of 8.
running batch 5...
done with 300 out of 434 rows in 6.0 seconds,
ran batch 5 of 8.
running batch 6...
done with 350 out of 434 rows in 5.98 seconds,
ran batch 6 of 8.
running batch 7...
done with 400 out of 434 rows in 4.69 seconds,
ran batch 7 of 8.
running batch 8...
no page token needed.
error (hopefully out of rows to query).
query finished in 59.9 total seconds.


In [6]:
geo_df = pd.DataFrame(geo)
geo_df.columns = geo_var
geo_df['flow_tmsp'] = pd.to_datetime(geo_df.flow_tmsp)
geo_df['premise_version'] = pd.to_datetime(geo_df.premise_version)
geo_df['latitude'] = pd.to_numeric(geo_df.latitude)
geo_df['longitude'] = pd.to_numeric(geo_df.longitude)
geo_df['geopoint_version'] = pd.to_datetime(geo_df.geopoint_version)
geo_df

,datasource_id,flow_tmsp,zip,premise_version,latitude,longitude,geopoint_version
0,ERCOT_ONCOR_10443720006444378,2019-07-19 04:55:29.193854,76543,2019-07-18 11:38:26.944058,31.113850,-97.702119,2019-07-18 11:38:26.944216
1,ERCOT_ONCOR_10443720007769008,2019-07-19 04:53:40.548727,76549,2019-07-18 11:38:31.532028,31.103194,-97.798050,2019-07-18 11:38:31.532290
2,PJM_BGE_8362930297,2019-07-19 14:39:13.831490,20794,2019-06-29 07:10:02.488668,39.169593,-76.785947,2019-06-29 07:10:02.488800
3,NEPOOL_PSNH_56119465086_985601009,2019-07-19 04:53:43.017792,03045,2019-07-18 11:38:29.263080,43.014408,-71.562076,2019-07-18 11:38:29.263317
4,PJM_BGE_2767448406,2019-07-19 14:44:25.074727,21230,2019-06-29 07:10:36.652858,39.249829,-76.645738,2019-06-29 07:10:36.652979
5,PJM_BGE_3931866968,2019-07-19 10:00:29.601324,21228,2019-07-19 09:55:27.964972,39.284529,-76.755986,2019-07-19 09:55:27.965082
6,PJM_DPL_0550117429257001616464,2019-07-19 14:43:10.578496,21643,2019-07-02 08:13:27.717165,38.630016,-75.858751,2019-07-02 08:13:27.717311
7,ERCOT_ONCOR_10443720001482952,2019-07-19 04:55:09.114261,76541,2019-07-18 11:34:26.033822,31.123675,-97.718379,2019-07-18 11:34:26.034431
8,PJM_BGE_8774854390,2019-07-19 14:39:07.131677,21040,2019-06-29 07:09:45.477326,39.434049,-76.316238,2019-06-29 07:09:45.477457
9,ERCOT_ONCOR_10443720003417908,2019-07-19 09:37:16.316513,75042,2019-07-19 09:31:42.779284,32.909691,-96.664165,2019-07-19 09:31:42.779610


## Accounts in PR Query

In [48]:
acct_query = "SELECT datasource_id FROM tags, UNNEST (properties) props WHERE props.key = 'prnumber' AND props.value = '1-IFF3XL'"

pr_result, pr_var = large_query(acct_query, 500)

starting query...
done with 500 out of 521 rows in 44.28 seconds.
running 1 more batches:
running batch 1...
no page token needed.
error (hopefully out of rows to query).


In [53]:
pr_result, pr_var = large_query(acct_query, 500)
pr_result_df = pd.DataFrame(pr_result)
pr_result_df = pr_result_df[[0]]
pr_result_df.columns = ['meters']
pr_result_df = pr_result_df.drop_duplicates()
pr_result_df

,meters
0,ERCOT_ONCOR_10443720002043653
1,ERCOT_AEPCPL_10032789409884049
2,ERCOT_CPT_1008901024901392150118
3,ERCOT_TNMP_10400519038300001
4,ERCOT_ONCOR_10443720001950770
